In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [ ]:
import os
import zipfile
import requests
import tensorflow as tf

DESIRED_ACCURACY = 0.999

path =  os.path.join('dataset') 
filename = "happy-or-sad.zip"
filepath = os.path.join(path, filename)

if not os.path.exists(path):
  os.makedirs(path)

if not os.path.exists(filepath):
  url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip"
  zfile = requests.get(url)
  open(filepath, 'wb').write(zfile.content)

if not os.path.exists('dataset/happy-or-sad'):
  zip_ref = zipfile.ZipFile("dataset/happy-or-sad.zip", 'r')
  zip_ref.extractall("dataset/happy-or-sad")
  zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  # Your Code
  def on_epoch_end(self,epoch,logs={}):
    if logs.get("accuracy")>DESIRED_ACCURACY:
      print("Reached Desired Accuracy of 99.9%")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
# Your Code Here
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation= 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation= 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer = RMSprop(lr=0.001),
              metrics=['accuracy'])

In [ ]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        'dataset/happy-or-sad',
        target_size=(150,150),
        batch_size = 4,
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

In [ ]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
      train_generator,
      steps_per_epoch=20,
      epochs=15,
      verbose=1,
      callbacks=callbacks)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

### Cleanup


In [ ]:
import os
import signal

datapaths = [os.path.join(path,'happy-or-sad')]
for datapath in datapaths:
    dirs = os.listdir(datapath)
    for dir in dirs:
        dirpath = os.path.join(datapath,dir)
        files = os.listdir(dirpath)
        for file in files:
            file_path = os.path.join(dirpath, file)  
            print(file_path)  
            os.remove(file_path)
        os.rmdir(dirpath)
    os.rmdir(datapath)
os.remove(filepath)
os.rmdir(path)

# Uncomment on colab
#os.kill(os.getpid(), signal.SIGKILL)